In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
import torch

In [3]:
device_name = torch.cuda.get_device_name(0)

torch_device_name = "cpu" if "AMD Radeon RX 580 2048SP" in device_name else "cuda"

print(f"Arquitetura de dispositivo: '{device_name}' e nome do dispositivo: '{torch_device_name}'" )

Arquitetura de dispositivo: 'AMD Radeon RX 580 2048SP' e nome do dispositivo: 'cpu'


In [21]:
class CPSolver:
    def __init__(self, model_path="solver", torch_device_name=torch_device_name):
        self.model = T5ForConditionalGeneration.from_pretrained(model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        self.device = torch.device(torch_device_name)
        self.model.to(self.device)

    def solve(self, problem_statement):
        # Preprocessamento do input para que o modelo possa entender melhor e gerar uma saída mais precisa.
        # Testamos vários tipos de strings de entrada, as menores tendem a criar saídas menos precisas.
        input_text = f"Generate the step-by-step programming for the problem: {problem_statement[:9000]}"

        inputs = self.tokenizer(
            input_text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=512,
            num_beams=4,
            early_stopping=False,
            length_penalty=-0.5,
            no_repeat_ngram_size=1,
            temperature=0.4,
            top_k=50,
            top_p=0.95,
            repetition_penalty=2.0,
            num_return_sequences=2,
            eos_token_id=self.tokenizer.eos_token_id,
            forced_eos_token_id=None

        )

        candidates = [self.tokenizer.decode(seq, skip_specials=True) for seq in outputs]
        return max(candidates, key=lambda x: len(x.split()))

In [22]:
solver = CPSolver(model_path='models/base-light')

In [37]:
problems = [
  "You are given two positive integers and In one move you can increase by replace with Your task is to find the minimum number of moves you need to do in order to make divisible by It is possible that you have to make moves as is already divisible by You have to answer independent test cases",
  "You are given two positive integers and In one move you can in the following way Choose any positive integer and replace with choose any positive integer and replace with You can perform as many such operations as you want You can choose the same numbers and in different moves Your task is to find the minimum number of moves required to obtain from It is guaranteed that you can always obtain from You have to answer independent test cases",
  "Polycarp has positive integers and He can perform the following operation Choose a integer and multiply of the integers or by Can Polycarp make it so that after performing the operation the sequence of three numbers forms an arithmetic progression Note that you the order of and Formally a sequence is called an arithmetic progression AP if there exists a number called common difference such that for all from to In this problem For example the following sequences are AP and The following sequences are not AP and You need to answer independent test cases ",
  "You are given a string consisting of lowercase English letters You must print the last character of the string",
  "There are N pigeons numbered from 1 to N, and there are N nests numbered from 1 to N Initially, pigeon i is in nest i for 1 less than N You are given Q queries, which you must process in order. There are two types of queries, each given in one of the following formats: Move P pigeon to nest H, Output the number of nests that contain more than one pigeon.",
  "Let s call a permutation of length if the condition holds for all Recall that the permutation is the array of length which contains each integer from to exactly once Your task is for a given number print anti Fibonacci permutations of length ",
  "Given a positive integer find integers not necessary distinct such that all these integers are strictly greater than and their product is equal to ",
  "You are given three positive integers and You have to construct a string of length consisting of lowercase Latin letters such that of length has distinct letters It is guaranteed that the answer exists You have to answer independent test cases Recall that the substring is the string and its length is In this problem you are only interested in substrings of length "
]

In [38]:
problem = problems[7]
solution = solver.solve(problem)

/usr/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [8]:
import re

In [15]:
def format_str(s):
    return re.sub(r'(?=[A-Z])', '\n', s)

In [39]:
print("STATEMENT:")
print(format_str(problem))
print()
print("EDITORIAL GERADO:")
print(format_str(solution))

STATEMENT:

You are given three positive integers and 
You have to construct a string of length consisting of lowercase 
Latin letters such that of length has distinct letters 
It is guaranteed that the answer exists 
You have to answer independent test cases 
Recall that the substring is the string and its length is 
In this problem you are only interested in substrings of length 

EDITORIAL GERADO:
<pad> 
Editorial: 
It is easy to see that we don t need any substrings of length 
You can use some binomial or even more primitive methods such as 
T
W
R
P and 
M
I
M
E 
To get the maximum possible answer you will firstly find three distinct consonants in each letter 
So for example there are two concatenations with letters 1 through 3 
In other words this number cannot be less than so it means exactly one 
Consequence has an equal sign 
If then does not exceed give us another negative integer 
The second problem works best when using binary calculus 
On average every decimal tree must con